# [Prime pair sets](https://projecteuler.net/problem=60)

> The primes 3, 7, 109, and 673, are quite remarkable. By taking any two primes and concatenating them in any order the result will always be prime. For example, taking 7 and 109, both 7109 and 1097 are prime. The sum of these four primes, 792, represents the lowest sum for a set of four primes with this property.
>
> Find the lowest sum for a set of five primes for which any two primes concatenate to produce another prime.

I’ll be relying on the [Primes.jl](https://juliamath.github.io/Primes.jl/stable/) package from [Julia Math](https://github.com/JuliaMath). It provides functions for generating and identifying prime numbers. Start by `using` it.

In [1]:
using Primes

primes(3, 11) # the primes from 3 to 11, inclusive

4-element Array{Int64,1}:
  3
  5
  7
 11

I’ll consider a grid with the odd primes as the indices and their concatenations as the entries:

|  ⋆ | 3   | 5   | 7   | 11   |
|----|-----|-----|-----|------|
| 3  | 33  | 35  | 37  | 311  |
| 5  | 53  | 55  | 57  | 511  |
| 7  | 73  | 75  | 77  | 711  |
| 11 | 113 | 115 | 117 | 1111 |

Odd primes because anything concatenated with ‘2’ on the right will be even, so not prime. I think anything on the diagonal ($n \star n$) will be a multiple of $11$, but I should prove that… (doesn’t matter for this problem because we are only interested in the concatenation of two primes, not a prime with itself) A pair of indices $i$ and $j$ meet the criteria if both $\mathbf{A}_{ij}$ and $\mathbf{A}_{ji}$ are prime, where $\mathbf{A}$ is that grid.

I’m using `⋆` (`\star`) as a concatenation operator. That will need to be defined:

In [2]:
# ⋆(a::Integer, b::Integer) = parse(Int, "$a$b") — too slow!
⋆(a::Integer, b::Integer) = a * 10^ndigits(b) + b
3⋆7

37

## Implementation Discovery

If we use the primes themselves as the indices, the array will be pretty sparse. (*comment about density of primes?*) We could perhaps use $i$ to refer to the $i$th prime, but maybe its better to start with a Dictionary with a tuple of primes as the key and their concatenation as the value. Entries where the elements of the tuple equal each other (*the diagonal*) should be excluded.

In [3]:
Dict((p, q) => p ⋆ q for p ∈ primes(3, 11), q ∈ primes(3, 11) if p ≠ q)

Dict{Tuple{Int64,Int64},Int64} with 12 entries:
  (11, 5) => 115
  (11, 3) => 113
  (7, 11) => 711
  (3, 11) => 311
  (7, 5)  => 75
  (7, 3)  => 73
  (5, 11) => 511
  (11, 7) => 117
  (3, 7)  => 37
  (5, 3)  => 53
  (3, 5)  => 35
  (5, 7)  => 57

We’re not really interested in the concatenation of the pairs, just whether it is prime or not:

In [4]:
Dict((p, q) => isprime(p ⋆ q) for p ∈ primes(3, 11), q ∈ primes(3, 11) if p ≠ q)

Dict{Tuple{Int64,Int64},Bool} with 12 entries:
  (11, 5) => false
  (11, 3) => true
  (7, 11) => false
  (3, 11) => true
  (7, 5)  => false
  (7, 3)  => true
  (5, 11) => false
  (11, 7) => false
  (3, 7)  => true
  (5, 3)  => true
  (3, 5)  => false
  (5, 7)  => false

Actually, we’re not interested in the pairs whose concatenation isn’t prime:

In [5]:
Dict(filter(x -> isprime(x.second), [(p, q) => p ⋆ q for p ∈ primes(3, 11), q ∈ primes(3, 11) if p ≠ q]))

Dict{Tuple{Int64,Int64},Int64} with 5 entries:
  (11, 3) => 113
  (3, 7)  => 37
  (5, 3)  => 53
  (7, 3)  => 73
  (3, 11) => 311

Maybe we don’t need a Dictionary at all, just a Set of tuples:

In [6]:
Set((p, q) for p ∈ primes(3, 11), q ∈ primes(3, 11) if p ≠ q && isprime(p ⋆ q))

Set{Tuple{Int64,Int64}} with 5 elements:
  (11, 3)
  (3, 7)
  (5, 3)
  (7, 3)
  (3, 11)

Now we can identify tuples where both ($p$, $q$) and ($q$, $p$) are in the set. Let’s keep just pairs where $p < q$, since one now implies the other.

In [7]:
P = primes(3, 673) # no need to generate this every time 673
A = Set((p, q) for p ∈ P, q ∈ P if p < q && isprime(p ⋆ q) && isprime(q ⋆ p))
length(A), length(P)

(418, 121)

In [8]:
function find4s(P, A)
    C = Set()
    l = 0
    for p₁ ∈ P, p₂ ∈ P, p₃ ∈ P, p₄ ∈ P
        if p₁ < p₂ < p₃ < p₄
            l += 1
            if (p₁, p₂) ∈ A && (p₁, p₃) ∈ A && (p₁, p₄) ∈ A && 
               (p₂, p₃) ∈ A && (p₂, p₄) ∈ A && (p₃, p₄) ∈ A
                println("Found: $p₁, $p₂, $p₃, $p₄")
                push!(C, Set([p₁, p₂, p₃, p₄]))
            end
        end
    end
    print(" did $l iterations ")
    C
end

find4s (generic function with 1 method)

In [9]:
@time find4s(P, A)

Found: 3, 7, 109, 673
 did 8495410 iterations   0.447990 seconds (250.07 k allocations: 12.430 MiB, 1.45% gc time)


Set{Any} with 1 element:
  Set([7, 673, 3, 109])

## Searching For More

So far, we’ve been identifying things we already knew existed in a fixed search space. To answer the problem, we’ll need to search an increasing space until we find what we're looking for, trying to avoid doing expensive computations more than needed.

In [10]:
function pairs!(A::Set{Tuple{Int,Int}}, P::Set{Int64})
  q = maximum(P)
  union!(A, Set((p, q) for p ∈ P if p < q && isprime(p ⋆ q) && isprime(q ⋆ p)))
end

function threes!(B::Set{Tuple{Int,Int,Int}}, A::Set{Tuple{Int,Int}}, p₃::Int)
  for (p₁, p₂) ∈ A
    if p₁ < p₂ < p₃
      (p₁, p₃) ∈ A && (p₂, p₃) ∈ A && push!(B, (p₁, p₂, p₃))
    end
  end
end

function fours!(C::Set{Tuple{Int,Int,Int,Int}}, B::Set{Tuple{Int,Int,Int}}, A::Set{Tuple{Int,Int}}, p₄::Int)
  for (p₁, p₂, p₃) ∈ B
    if p₁ < p₂ < p₃ < p₄
      (p₁, p₄) ∈ A && (p₂, p₄) ∈ A && (p₃, p₄) ∈ A && push!(C, (p₁, p₂, p₃, p₄))
    end
  end
end

function fives!(D::Set{Tuple{Int,Int,Int,Int,Int}}, C::Set{Tuple{Int,Int,Int,Int}}, A::Set{Tuple{Int,Int}}, p₅::Int)
  for (p₁, p₂, p₃, p₄) ∈ C
    if p₁ < p₂ < p₃ < p₄ < p₅
      (p₁, p₅) ∈ A && (p₂, p₅) ∈ A && (p₃, p₅) ∈ A && (p₄, p₅) ∈ A && push!(D, (p₁, p₂, p₃, p₄, p₅))
    end
  end
end

function problem60()
  P = Set{Int64}([3])
  A = Set{Tuple{Int,Int}}()
  B = Set{Tuple{Int,Int,Int}}()
  C = Set{Tuple{Int,Int,Int,Int}}()
  D = Set{Tuple{Int,Int,Int,Int,Int}}()
  while length(D) == 0
    p = nextprime(maximum(P), 2)
    push!(P, p)
    pairs!(A, P)
    threes!(B, A, p)
    fours!(C, B, A, p)
    fives!(D, C, A, p)
    # println(p, "\t", length(A), "\t", length(B), "\t", length(C), "\t", length(D))
  end
  println("D: $D")
  println("min ΣDᵢ: ", minimum(sum, D))
end

problem60 (generic function with 1 method)

I’m building up five sets: odd primes, pairs of primes whose concatenations are prime, and then sets of 3, 4, and 5 primes, any two of which’s concatenations are prime. Each set it built from the smaller ones as we add prime by prime. Once there is at least one entry in the five-set, we stop and find the element of that set with the smallest sum of entries. In this case there is just one when we stop. I’m confident that this is the one with the smallest sum because of the way I build up the sets. We always add the next prime, so set elements contain the smallest primes available. 

In [12]:
@time problem60()

D: Set([(13, 5197, 5701, 6733, 8389)])
min ΣDᵢ: 26033
  0.356895 seconds (208.02 k allocations: 6.572 MiB)


## Discussion

Different implementation choices affected performance significantly in some cases. Concatenating primes by converting them to strings and then parsing the result was measurably slower than working with integers. The type annotations make a big difference. (For all of these, I used the `@time` macro and did a/b testing.) I tried only running `fives!` when set `C` changed, only running `fours!` when set `B` changed, etc., but it didn’t make enough difference to justify the added visual clutter. Of course, what really made the biggest difference was bulding up the sets as we add primes and not computing things from scratch with each iteration.